In [ ]:
# OCR PDF Processing Code
# Complete implementation for reading OCR scanned PDF files using PIL and related libraries

# Required Libraries Installation:
# pip install Pillow pytesseract pdf2image opencv-python numpy

import os
import cv2
import json
import time
import numpy as np
import pytesseract
from PIL import Image, ImageEnhance, ImageFilter, ImageOps
from pdf2image import convert_from_path
from datetime import datetime

# Windows users may need to specify tesseract path
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# ========== BASIC PDF OCR FUNCTIONS ==========

def simple_pdf_ocr(pdf_path):
    """
    Basic PDF OCR with minimal preprocessing
    """
    # Convert PDF to images
    pages = convert_from_path(pdf_path, dpi=300)
    
    extracted_text = []
    
    for page_num, page in enumerate(pages):
        # Convert to PIL Image and perform OCR
        text = pytesseract.image_to_string(page)
        extracted_text.append(f"=== Page {page_num + 1} ===\n{text}\n")
    
    return "\n".join(extracted_text)

def enhanced_pdf_ocr(pdf_path, output_folder=None):
    """
    Enhanced PDF OCR with PIL image preprocessing
    """
    # Convert PDF to images with high DPI for better quality
    pages = convert_from_path(pdf_path, dpi=300, fmt='png')
    
    extracted_text = []
    
    for page_num, page in enumerate(pages):
        # Save original if output folder specified
        if output_folder:
            os.makedirs(output_folder, exist_ok=True)
            page.save(f"{output_folder}/page_{page_num + 1}_original.png")
        
        # Preprocess the image
        processed_page = preprocess_image_for_ocr(page)
        
        # Save processed image
        if output_folder:
            processed_page.save(f"{output_folder}/page_{page_num + 1}_processed.png")
        
        # Perform OCR with custom configuration
        custom_config = r'--oem 3 --psm 6'
        text = pytesseract.image_to_string(processed_page, config=custom_config)
        
        extracted_text.append(f"=== Page {page_num + 1} ===\n{text}\n")
    
    return "\n".join(extracted_text)

# ========== IMAGE PREPROCESSING FUNCTIONS ==========

def preprocess_image_for_ocr(image):
    """
    Preprocess image using PIL to improve OCR accuracy
    """
    # Convert to grayscale
    if image.mode != 'L':
        image = image.convert('L')
    
    # Enhance contrast
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(2.0)
    
    # Enhance sharpness
    enhancer = ImageEnhance.Sharpness(image)
    image = enhancer.enhance(1.5)
    
    # Apply noise reduction
    image = image.filter(ImageFilter.MedianFilter(size=3))
    
    # Convert to numpy array for advanced processing
    img_array = np.array(image)
    
    # Apply adaptive threshold for better text separation
    img_array = cv2.adaptiveThreshold(
        img_array, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
        cv2.THRESH_BINARY, 11, 2
    )
    
    # Convert back to PIL Image
    return Image.fromarray(img_array)

def advanced_image_preprocessing(image):
    """
    Advanced image preprocessing for better OCR results
    """
    # Convert to grayscale if needed
    if image.mode != 'L':
        image = image.convert('L')
    
    # Convert to numpy array
    img_array = np.array(image)
    
    # 1. Noise reduction
    img_array = cv2.medianBlur(img_array, 3)
    
    # 2. Gaussian blur to smooth
    img_array = cv2.GaussianBlur(img_array, (1, 1), 0)
    
    # 3. Adaptive thresholding for better text separation
    img_array = cv2.adaptiveThreshold(
        img_array, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
        cv2.THRESH_BINARY, 11, 2
    )
    
    # 4. Morphological operations to clean up
    kernel = np.ones((1, 1), np.uint8)
    img_array = cv2.morphologyEx(img_array, cv2.MORPH_CLOSE, kernel)
    img_array = cv2.morphologyEx(img_array, cv2.MORPH_OPEN, kernel)
    
    # Convert back to PIL Image
    processed_image = Image.fromarray(img_array)
    
    # 5. PIL-specific enhancements
    # Enhance contrast
    enhancer = ImageEnhance.Contrast(processed_image)
    processed_image = enhancer.enhance(1.2)
    
    # Enhance sharpness
    enhancer = ImageEnhance.Sharpness(processed_image)
    processed_image = enhancer.enhance(1.5)
    
    return processed_image

def rotate_image_if_needed(image):
    """
    Detect and correct image rotation using PIL
    """
    # Convert to numpy for rotation detection
    img_array = np.array(image)
    
    # Use OpenCV to detect text orientation
    try:
        coords = np.column_stack(np.where(img_array > 0))
        angle = cv2.minAreaRect(coords)[-1]
        
        if angle < -45:
            angle = -(90 + angle)
        else:
            angle = -angle
            
        # Rotate image using PIL
        if abs(angle) > 1:  # Only rotate if significant angle
            image = image.rotate(angle, expand=True, fillcolor='white')
    except:
        pass  # Skip rotation if detection fails
    
    return image

def process_different_image_types(image):
    """
    Handle different types of scanned documents
    """
    # Get image statistics
    extrema = image.getextrema()
    
    if image.mode == 'RGB':
        # Color image - convert to grayscale smartly
        # Check if it's already mostly grayscale
        r_range = extrema[0][1] - extrema[0][0]
        g_range = extrema[1][1] - extrema[1][0]
        b_range = extrema[2][1] - extrema[2][0]
        
        if max(r_range, g_range, b_range) < 50:
            # Already mostly grayscale
            image = image.convert('L')
        else:
            # Convert using luminance
            image = image.convert('L')
    
    # Handle very dark or very light images
    if image.mode == 'L':
        min_val, max_val = extrema
        
        if max_val - min_val < 50:  # Low contrast
            # Enhance contrast significantly
            enhancer = ImageEnhance.Contrast(image)
            image = enhancer.enhance(3.0)
        
        # Auto-level using PIL
        image = ImageOps.autocontrast(image)
    
    return image

# ========== OCR CONFIGURATION FUNCTIONS ==========

def ocr_with_custom_config(image, lang='eng'):
    """
    OCR with custom Tesseract configuration
    """
    # Different PSM modes for different document types
    configs = {
        'single_block': r'--oem 3 --psm 6',  # Single uniform block
        'single_line': r'--oem 3 --psm 7',   # Single text line
        'single_word': r'--oem 3 --psm 8',   # Single word
        'sparse_text': r'--oem 3 --psm 11',  # Sparse text
        'auto': r'--oem 3 --psm 3'           # Auto page segmentation
    }
    
    results = {}
    
    for config_name, config in configs.items():
        try:
            text = pytesseract.image_to_string(
                image, 
                lang=lang, 
                config=config
            )
            results[config_name] = text
        except Exception as e:
            results[config_name] = f"Error: {str(e)}"
    
    return results

def get_ocr_confidence(image):
    """
    Get OCR confidence scores
    """
    data = pytesseract.image_to_data(image, output_type=pytesseract.Output.DICT)
    
    confidences = [int(conf) for conf in data['conf'] if int(conf) > 0]
    
    if confidences:
        avg_confidence = sum(confidences) / len(confidences)
        return avg_confidence, confidences
    else:
        return 0, []

def multilingual_ocr(image, languages=['eng', 'spa', 'fra']):
    """
    Try OCR with multiple languages
    """
    results = {}
    
    for lang in languages:
        try:
            text = pytesseract.image_to_string(image, lang=lang)
            confidence, _ = get_ocr_confidence(image)
            results[lang] = {
                'text': text,
                'confidence': confidence
            }
        except Exception as e:
            results[lang] = {
                'text': '',
                'confidence': 0,
                'error': str(e)
            }
    
    # Return result with highest confidence
    best_lang = max(results.keys(), key=lambda x: results[x]['confidence'])
    return results[best_lang]['text'], results

# ========== SPECIALIZED DOCUMENT PROCESSING ==========

def extract_table_from_image(image):
    """
    Extract tables from images using PIL preprocessing
    """
    # Convert to grayscale
    if image.mode != 'L':
        image = image.convert('L')
    
    # Convert to numpy array
    img_array = np.array(image)
    
    # Detect horizontal and vertical lines
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (25, 1))
    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 25))
    
    # Detect lines
    horizontal_lines = cv2.morphologyEx(img_array, cv2.MORPH_OPEN, horizontal_kernel)
    vertical_lines = cv2.morphologyEx(img_array, cv2.MORPH_OPEN, vertical_kernel)
    
    # Combine lines
    table_structure = cv2.addWeighted(horizontal_lines, 0.5, vertical_lines, 0.5, 0.0)
    
    # Convert back to PIL
    table_image = Image.fromarray(table_structure)
    
    # OCR with table-specific configuration
    table_text = pytesseract.image_to_string(
        table_image, 
        config=r'--oem 3 --psm 6 -c preserve_interword_spaces=1'
    )
    
    return table_text

def process_handwritten_text(image):
    """
    Preprocess images for handwritten text OCR
    """
    # Convert to grayscale
    if image.mode != 'L':
        image = image.convert('L')
    
    # Enhance contrast more aggressively for handwriting
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(2.5)
    
    # Apply slight blur to smooth pen strokes
    image = image.filter(ImageFilter.GaussianBlur(radius=0.5))
    
    # Use specialized OCR configuration for handwriting
    handwritten_config = r'--oem 3 --psm 8 -c tessedit_char_whitelist=0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'
    
    text = pytesseract.image_to_string(image, config=handwritten_config)
    
    return text

# ========== MAIN PDF PROCESSOR CLASS ==========

class PDFOCRProcessor:
    def __init__(self, tesseract_path=None):
        if tesseract_path:
            pytesseract.pytesseract.tesseract_cmd = tesseract_path
        
        self.processed_pages = []
        self.processing_stats = {}
    
    def process_pdf(self, pdf_path, output_dir=None, languages=['eng']):
        """
        Complete PDF processing pipeline
        """
        start_time = datetime.now()
        
        # Create output directory
        if output_dir:
            os.makedirs(output_dir, exist_ok=True)
        
        # Convert PDF to images
        try:
            pages = convert_from_path(pdf_path, dpi=300, fmt='png')
        except Exception as e:
            return f"Error converting PDF: {str(e)}"
        
        extracted_text = []
        page_stats = []
        
        for page_num, page in enumerate(pages):
            print(f"Processing page {page_num + 1}/{len(pages)}")
            
            # Process single page
            page_result = self.process_single_page(
                page, page_num + 1, output_dir, languages
            )
            
            extracted_text.append(page_result['text'])
            page_stats.append(page_result['stats'])
        
        # Compile results
        processing_time = datetime.now() - start_time
        
        results = {
            'pdf_path': pdf_path,
            'total_pages': len(pages),
            'processing_time': str(processing_time),
            'extracted_text': "\n".join(extracted_text),
            'page_stats': page_stats
        }
        
        # Save results
        if output_dir:
            with open(f"{output_dir}/ocr_results.json", 'w') as f:
                json.dump(results, f, indent=2)
            
            with open(f"{output_dir}/extracted_text.txt", 'w') as f:
                f.write(results['extracted_text'])
        
        return results
    
    def process_single_page(self, page, page_num, output_dir, languages):
        """
        Process a single page with comprehensive preprocessing
        """
        # Save original
        if output_dir:
            page.save(f"{output_dir}/page_{page_num:03d}_original.png")
        
        # Preprocess image
        processed_page = self.comprehensive_preprocessing(page)
        
        # Save processed
        if output_dir:
            processed_page.save(f"{output_dir}/page_{page_num:03d}_processed.png")
        
        # Perform OCR
        ocr_results = multilingual_ocr(processed_page, languages)
        text, lang_results = ocr_results
        
        # Calculate statistics
        stats = {
            'page_number': page_num,
            'original_size': page.size,
            'processed_size': processed_page.size,
            'character_count': len(text),
            'word_count': len(text.split()),
            'language_results': lang_results
        }
        
        return {
            'text': f"=== Page {page_num} ===\n{text}\n",
            'stats': stats
        }
    
    def comprehensive_preprocessing(self, image):
        """
        Apply all preprocessing techniques
        """
        # Step 1: Basic PIL preprocessing
        image = process_different_image_types(image)
        
        # Step 2: Rotation correction
        image = rotate_image_if_needed(image)
        
        # Step 3: Advanced preprocessing
        image = advanced_image_preprocessing(image)
        
        return image

# ========== ERROR HANDLING AND OPTIMIZATION ==========

def safe_pdf_ocr(pdf_path, max_retries=3):
    """
    PDF OCR with comprehensive error handling
    """
    for attempt in range(max_retries):
        try:
            # Check if file exists and is readable
            if not os.path.exists(pdf_path):
                raise FileNotFoundError(f"PDF file not found: {pdf_path}")
            
            # Check file size
            file_size = os.path.getsize(pdf_path)
            if file_size > 50 * 1024 * 1024:  # 50MB limit
                print(f"Warning: Large file ({file_size/1024/1024:.1f}MB), processing may be slow")
            
            # Convert PDF
            pages = convert_from_path(pdf_path, dpi=200)  # Lower DPI for large files
            
            if not pages:
                raise ValueError("No pages found in PDF")
            
            # Process pages
            extracted_text = []
            for page_num, page in enumerate(pages):
                try:
                    # Process with timeout-like behavior
                    processed_page = preprocess_image_for_ocr(page)
                    text = pytesseract.image_to_string(processed_page)
                    extracted_text.append(f"=== Page {page_num + 1} ===\n{text}\n")
                except Exception as page_error:
                    print(f"Error processing page {page_num + 1}: {str(page_error)}")
                    extracted_text.append(f"=== Page {page_num + 1} ===\n[Error processing page]\n")
            
            return "\n".join(extracted_text)
            
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {str(e)}")
            if attempt == max_retries - 1:
                return f"Failed to process PDF after {max_retries} attempts: {str(e)}"
            
            # Wait before retry
            time.sleep(2 ** attempt)

def process_large_pdf(pdf_path, batch_size=5):
    """
    Process large PDFs in batches to manage memory
    """
    # Get total page count first
    import subprocess
    try:
        result = subprocess.run(['pdfinfo', pdf_path], capture_output=True, text=True)
        total_pages = int([line for line in result.stdout.split('\n') if 'Pages:' in line][0].split()[-1])
    except:
        # Fallback: convert entire PDF to get page count
        pages = convert_from_path(pdf_path, dpi=72)  # Low DPI for counting
        total_pages = len(pages)
        del pages
    
    extracted_text = []
    
    # Process in batches
    for start_page in range(1, total_pages + 1, batch_size):
        end_page = min(start_page + batch_size - 1, total_pages)
        
        # Convert only current batch
        pages = convert_from_path(
            pdf_path, 
            dpi=300, 
            first_page=start_page, 
            last_page=end_page
        )
        
        # Process batch
        for page_num, page in enumerate(pages):
            processed_page = preprocess_image_for_ocr(page)
            text = pytesseract.image_to_string(processed_page)
            extracted_text.append(f"=== Page {start_page + page_num} ===\n{text}\n")
        
        # Clear memory
        del pages
        
        print(f"Processed pages {start_page}-{end_page}/{total_pages}")
    
    return "\n".join(extracted_text)

# ========== USAGE EXAMPLES ==========

def main():
    """
    Example usage of the OCR PDF processing functions
    """
    # Example 1: Simple PDF OCR
    print("=== Simple PDF OCR ===")
    try:
        simple_text = simple_pdf_ocr('example.pdf')
        print(simple_text[:500] + "..." if len(simple_text) > 500 else simple_text)
    except Exception as e:
        print(f"Error in simple OCR: {e}")
    
    # Example 2: Enhanced PDF OCR with preprocessing
    print("\n=== Enhanced PDF OCR ===")
    try:
        enhanced_text = enhanced_pdf_ocr('example.pdf', 'output_images')
        print(enhanced_text[:500] + "..." if len(enhanced_text) > 500 else enhanced_text)
    except Exception as e:
        print(f"Error in enhanced OCR: {e}")
    
    # Example 3: Complete PDF processing pipeline
    print("\n=== Complete PDF Processing ===")
    try:
        processor = PDFOCRProcessor()
        results = processor.process_pdf(
            'example.pdf',
            output_dir='ocr_output',
            languages=['eng', 'spa']
        )
        print(f"Processed {results['total_pages']} pages")
        print(f"Processing time: {results['processing_time']}")
        print(f"Total text length: {len(results['extracted_text'])} characters")
    except Exception as e:
        print(f"Error in complete processing: {e}")
    
    # Example 4: Safe PDF OCR with error handling
    print("\n=== Safe PDF OCR ===")
    try:
        safe_text = safe_pdf_ocr('example.pdf')
        print(safe_text[:500] + "..." if len(safe_text) > 500 else safe_text)
    except Exception as e:
        print(f"Error in safe OCR: {e}")

if __name__ == "__main__":
    main()
